In [ ]:
# Import the packages for this lab
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Importion Random forest regressor
from sklearn.ensemble import RandomForestRegressor

# optimization package
import cvxpy as cp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_freq = "/content/drive/MyDrive/MIE368 Group Project/Data/Nov 29 2022 New Features GSOD/Nov 29 2022 state_freq_data_dates_adjusted.csv"
path_costs = "/content/drive/My Drive/MIE368 Group Project/Data/state-cost-data1.csv"
path_region = "/content/drive/My Drive/MIE368 Group Project/Data/region_freq_data_dates_adjusted.csv"

# Data Cleaning

In [ ]:
df_raw =  pd.read_csv(path_freq)
df_raw = df_raw.drop(columns = "helper")
df = df_raw.dropna()
df = df.reset_index()
df = df.drop(columns = 'index')
df.head()

,year,state,drought,flooding,freeze,severe storm,tropical cyclone,wildfire,winter storm,Cost,...,STP,STP_ATTRIBUTES,VISIB,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,SNDP
0,1980,AL,1,0,0,0,0,0,0,2000.0,...,860.847395,5.767130,9.331530,16.462221,8.126788,16.434746,79.467499,854.755168,73.564098,998.896113
1,1981,AL,0,0,0,1,0,0,0,500.0,...,865.555243,5.005061,9.420876,15.965681,6.393136,15.952870,86.733940,851.583331,77.522268,999.900000
2,1982,AL,0,0,0,1,0,0,1,100.0,...,865.387214,5.423220,10.137709,15.685759,6.195882,15.680805,81.693096,850.118204,73.196780,996.347972
3,1983,AL,1,1,1,0,0,0,0,500.0,...,880.665251,5.332420,8.892329,15.979452,7.389391,15.956925,87.680913,823.871476,71.464551,990.060289
4,1984,AL,0,0,0,0,0,0,0,0.0,...,875.592727,5.137129,9.856549,16.611737,7.576116,16.593005,60.235724,834.220138,72.987912,994.212791


# Make states dummy variables

In [ ]:
# Convert categorical data into "dummy" variables
df_states = pd.get_dummies(df, columns=['state'])
df_states.head()

,year,drought,flooding,freeze,severe storm,tropical cyclone,wildfire,winter storm,Cost,Avg_Tmp,...,state_PA,state_RI,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WI,state_WY
0,1980,1,0,0,0,0,0,0,2000.0,61.9,...,0,0,0,0,0,0,0,0,0,0
1,1981,0,0,0,1,0,0,0,500.0,62.3,...,0,0,0,0,0,0,0,0,0,0
2,1982,0,0,0,1,0,0,1,100.0,63.3,...,0,0,0,0,0,0,0,0,0,0
3,1983,1,1,1,0,0,0,0,500.0,61.0,...,0,0,0,0,0,0,0,0,0,0
4,1984,0,0,0,0,0,0,0,0.0,62.6,...,0,0,0,0,0,0,0,0,0,0


# Prediction: Random Forest Regressor

In [ ]:
df_train = df_states[df_states.year<2018]
df_test = df_states[df_states.year>=2018]

# ########################################################
# ########### Drop highly correlated features ############
# ########################################################
# df_train = df_train.drop(columns = ["Avg_Tmp","Min_Tmp","Max_Tmp","Population","GDP","Personal Income","TEMP_ATTRIBUTES","DEWP_ATTRIBUTES","VISIB_ATTRIBUTES","WDSP_ATTRIBUTES","GUST"])
# df_test = df_test.drop(columns = ["Avg_Tmp","Min_Tmp","Max_Tmp","Population","GDP","Personal Income","TEMP_ATTRIBUTES","DEWP_ATTRIBUTES","VISIB_ATTRIBUTES","WDSP_ATTRIBUTES","GUST"])

# Define features to remove from the feature data (i.e., X)
drop_for_X = ['year','Cost']#,'TEMP_ATTRIBUTES']#,'DEWP_ATTRIBUTES','GUST','WDSP_ATTRIBUTES','VISIB_ATTRIBUTES']

# Partition the training data into features and target
X_train = df_train.drop(columns=drop_for_X)
y_train = df_train.Cost

# # Partition the testing data into features and target
X_test = df_test.drop(columns=drop_for_X)
y_test = df_test.Cost

cart_mdl = RandomForestRegressor(random_state=16)
# cart_mdl = RandomForestRegressor(n_estimators= 200, min_samples_split = 3, min_samples_leaf = 3, max_features='sqrt',max_depth =5, bootstrap = True, random_state=16)

cart_mdl.fit(X_train, y_train)
cart_mdl_predictions = cart_mdl.predict(X_test)

train_score = cart_mdl.score(X_train, y_train)
test_score = cart_mdl.score(X_test, y_test)
print(f'The train score is {train_score:.3f} and the test score is {test_score:.3f}')
#mean_absolute_percentage_error
print(np.mean(np.abs((y_test - cart_mdl_predictions) / cart_mdl_predictions)) * 100)

The train score is 0.825 and the test score is -0.471
93.57912682460778


# Optimization

In [ ]:
path_predictions = "/content/drive/MyDrive/MIE368 Group Project/5. Final Report/Predictions.csv"
predictions =  pd.read_csv(path_predictions)
predictions = predictions.set_index(predictions.state)
predictions = predictions.drop(columns = "state")

In [ ]:
calif = list(predictions.index).index("CA")
texas = list(predictions.index).index("TX")
new_york = list(predictions.index).index("NY")
pennsyl = list(predictions.index).index("PA")
illinois = list(predictions.index).index("IL")

In [ ]:
# 44 states

# The number of business chains that will be established in state i
x = cp.Variable(44, integer=True)  # x is a vector of integer variables of size 44.
obj = cp.Minimize(x@predictions.cost)

cons = [] #initialize constraint list
cons.append(cp.sum(x)==100)  # The total number of branches must be equal to 100
cons.append(x>=0) # The total number of branches in each state must be greater than 0.
cons.append(x<=5) # The total number of branches in each state cannot exceed 8

# At least two branches must be opened in each of the 5 states with the highest populations (California,Texas, New York, Pennsylvania, and Illinois).
# Although Florida is most populated after california, it is not in our dataset
cons.append(x[calif] >=2)
cons.append(x[texas] >=2)
cons.append(x[new_york] >=2)
cons.append(x[pennsyl] >=2)
cons.append(x[illinois] >=2)

# Put the model together
prob = cp.Problem(obj,cons)
prob.solve(verbose=False)  # verbose = True allows you to see the solution process
print('The objective value is {}.'.format(obj.value))  # gives you the objective value

The objective value is 94746.40416199999.


In [ ]:
#Extract the solution of the IP into a dataframe
x_np_array = x.value.astype(int)  # extract the x values as a np array
x_values = pd.Series(x_np_array, index = predictions.index)  # convert the np array to a Dataframe
selected = x_values[x_values >= 1].index.values  # get assignments

# Print outputs
print(x_values)
print(selected)

state
AL    5
AR    5
AZ    5
CA    2
CO    0
CT    0
DE    3
GA    0
IA    5
ID    5
IL    5
IN    5
KS    0
KY    0
LA    0
MA    0
MD    0
ME    0
MI    5
MN    0
MO    0
MS    0
MT    0
NC    0
ND    5
NE    5
NH    5
NJ    5
NM    5
NV    5
NY    5
OH    0
OK    0
OR    0
PA    5
RI    0
SD    0
TN    0
TX    5
UT    5
VA    0
VT    0
WI    5
WY    0
dtype: int64
['AL' 'AR' 'AZ' 'CA' 'DE' 'IA' 'ID' 'IL' 'IN' 'MI' 'ND' 'NE' 'NH' 'NJ'
 'NM' 'NV' 'NY' 'PA' 'TX' 'UT' 'WI']


In [ ]:
len(selected)

21